In [ ]:
from tm.trading_rules import SimpleMovingAverage
from tm import StockDataProvider
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
import math
import pandas as pd
%matplotlib inline
set_matplotlib_formats('svg')

In [ ]:
microsoft_data = StockDataProvider(ticker_name='MSFT')

In [ ]:
sma = SimpleMovingAverage(microsoft_data, days=50)

In [ ]:
start_day, end_day = ('01-01-2019', '07-01-2019')
fig, ax = plt.subplots()
ax.plot(microsoft_data.history['Close'].loc[start_day:end_day], label='Microsoft stock prices')
ax.plot(sma.calculate().loc[start_day:end_day], label='Simple 50 days moving average')

# Buy signals
buy_signals = sma.buy_signals().loc[start_day:end_day]
buy_points = pd.Series(data=map(lambda index, price: price if buy_signals.loc[index] == True else math.nan, microsoft_data.history['Close'].loc[start_day:end_day].index, microsoft_data.history['Close'].loc[start_day:end_day]),
                       index=microsoft_data.history['Close'].loc[start_day:end_day].index)
ax.scatter(x=buy_points.index, y=buy_points, color='green', marker='x', label='Buy Signal')

# Sell signals
sell_signals = sma.sell_signals().loc[start_day:end_day]
sell_points = pd.Series(data=map(lambda index, price: price if sell_signals.loc[index] == True else math.nan, microsoft_data.history['Close'].loc[start_day:end_day].index, microsoft_data.history['Close'].loc[start_day:end_day]),
                       index=microsoft_data.history['Close'].loc[start_day:end_day].index)
ax.scatter(x=sell_points.index, y=sell_points, color='red', marker='x', label='Sell Signal')

ax.legend()
fig.tight_layout()

In [ ]:
profits = sell_points[~sell_points.isna()].reset_index(drop=True) - buy_points[~buy_points.isna()].reset_index(drop=True)
profit_dollar = sum(profits.fillna(0))
print('Profit:', profit_dollar, '$')

In [ ]:
profit_percent = profit_dollar / buy_points[~buy_points.isna()].reset_index(drop=True)[0]
print('Profit', profit_percent * 100, '%')